In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path = '/home/seewon/mimic3-benchmarks/output'

FIGURE OUT WHAT FILES ARE

In [5]:
master_df = pd.read_csv('/home/seewon/mimic3-benchmarks/output/master_dataset.csv')

In [ ]:
df_train = pd.read_csv((os.path.join(path, 'train.csv')))
df_test = pd.read_csv((os.path.join(path, 'test.csv')))
df = pd.concat([df_train, df_test], ignore_index=True)

In [33]:
diff_cols = list(set(df.columns) - set(master_df.columns))

In [35]:
master_df = master_df.merge(df[diff_cols + ['stay_id']], on='stay_id', how='outer')

In [36]:
master_df.to_csv('/home/seewon/mimic3-benchmarks/output/master_dataset.csv', index=False)

GET ONLY LABELS

In [ ]:
master_df = pd.read_csv('/home/seewon/mimic3-benchmarks/output/master_dataset.csv')

In [43]:
if 'index' in master_df.columns:
    master_df = master_df.drop(columns=['index'])

In [ ]:
ID_COLS = ['subject_id', 'hadm_id', 'stay_id',]
DEMO_COLS = ['gender', 'race', 'anchor_age', 'anchor_year', 'age', 'ethnicity',]
LABELS_COLS = ['intime', 'outtime', 'outcome_icu_transfer_12h', 'intime_icu', 'time_to_icu_transfer',  'time_to_icu_transfer_hours',]

In [73]:
master_df[ID_COLS + LABELS_COLS].to_csv(os.path.join(path, 'label.csv'), index=False)

In [75]:
master_df[ID_COLS + DEMO_COLS].to_csv(os.path.join(path, 'demo.csv'), index=False)

CREATE DATASET

In [84]:
path = '/home/seewon/mimic3-benchmarks/data/mimiciv/3.1/ed'

In [172]:
stays_df = pd.read_csv(os.path.join(path, 'edstays.csv'))
vital_df = pd.read_csv(os.path.join(path, 'vitalsign.csv'))
triage_df = pd.read_csv(os.path.join(path, 'triage.csv'))
med_df = pd.read_csv(os.path.join(path, 'pyxis.csv'))

In [173]:
triage_df = triage_df.merge(stays_df[['stay_id', 'intime']], on='stay_id')
triage_df = triage_df.rename(columns={'intime': 'charttime'})

In [174]:
vital_df = pd.concat([triage_df, vital_df]).sort_values(['subject_id', 'charttime'])

In [176]:
vital_df.to_csv('/home/seewon/mimic3-benchmarks/data/transfer/vital_triage.csv', index=False)

In [177]:
import networkx as nx

med_df['gsn'] = med_df['gsn'].astype(str)
med_df['name'] = med_df['name'].astype(str)

G = nx.Graph()
edges = med_df[['gsn', 'name']].dropna().drop_duplicates().values.tolist()
G.add_edges_from(edges)


components = list(nx.connected_components(G))
node_to_group = {node: i for i, comp in enumerate(components) for node in comp}

In [178]:
med_df['group'] = med_df.apply(
    lambda row: node_to_group.get(row['gsn']) or node_to_group.get(row['name']),
    axis=1
)

In [180]:
med_df.to_csv('/home/seewon/mimic3-benchmarks/data/transfer/med.csv')

DEMOGRAPHICS

In [166]:
demo_df = pd.read_csv(f"/home/seewon/mimic3-benchmarks/data/transfer/demo.csv")

In [167]:
demo_df['gender_ind'] = pd.factorize(demo_df['gender'])[0]
demo_df['race_ind'] = pd.factorize(demo_df['race'])[0]

In [169]:
demo_df.to_csv("/home/seewon/mimic3-benchmarks/data/transfer/demo.csv", index=False)

SAMPLE TIME

In [2]:
label_df = pd.read_csv('/home/seewon/mimic3-benchmarks/data/transfer/label.csv')

In [3]:
label_df = label_df.drop(columns=['Trigger_time', 'Sample_time'])

In [4]:
label_df['intime'] = pd.to_datetime(label_df['intime'])
label_df['outtime'] = pd.to_datetime(label_df['outtime'])
label_df['intime_icu'] = pd.to_datetime(label_df['intime_icu'])

In [5]:
label_df = label_df[(label_df['outtime'] == label_df['intime_icu']) | label_df['intime_icu'].isna()]

In [6]:
label_df = label_df.drop(columns=['time_to_icu_transfer', 'time_to_icu_transfer_hours'])

First, filter to have one example per patient id

In [7]:
pos_df = label_df[~label_df['intime_icu'].isna()]
neg_df = label_df[label_df['intime_icu'].isna()]

In [28]:
pos_df = pos_df.drop_duplicates()
neg_df = neg_df.drop_duplicates()

In [ ]:
pos_df = pos_df[pos_df['intime'] == pos_df.groupby('subject_id')['intime'].transform('min')]

In [ ]:
neg_df = neg_df[~neg_df['subject_id'].isin(pos_df['subject_id'])]
neg_df = neg_df[neg_df['intime'] == neg_df.groupby('subject_id')['intime'].transform('min')]

In [48]:
pos_df['Trigger_time'] = (pos_df['outtime'] - pos_df['intime']).dt.total_seconds() / 3600
pos_df = pos_df[pos_df['Trigger_time'] >= 0.5]

In [71]:
deltas = pos_df['Trigger_time'].values
times = np.random.choice(deltas, size=int(len(neg_df) * 1.2), replace=True) + np.random.normal(0, 0.5, int(len(neg_df) * 1.2))

In [72]:
neg_df['Sample_time'] = [t for t in times if t >= 0.5][:len(neg_df)]

/tmp/ipykernel_1402567/1178699263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_df['Sample_time'] = [t for t in times if t >= 0.5][:len(neg_df)]


In [ ]:
df = pd.concat([pos_df, neg_df], ignore_index=True)
df['Time'] = df.apply(lambda x: x['Trigger_time'] if x['outcome_icu_transfer_12h'] else x['Sample_time'], axis=1)

In [79]:
df.to_csv('/home/seewon/mimic3-benchmarks/data/transfer/label.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Plot histograms normalized to fraction
ax = pos_df['Trigger_time'].plot(
    kind='hist', bins=1000, alpha=0.5, label='Trigger_time', density=True, figsize=(10, 6)
)
neg_df['Sample_time'].plot(
    kind='hist', bins=1000, alpha=0.5, label='Sample_time', density=True, ax=ax
)

# Labels and legend
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Fraction')
ax.set_title('Normalized Distribution of Trigger_time and Sample_time')
ax.legend()
ax.set_xlim(0, 100)

STRING VALUES

In [2]:
df = pd.read_csv('/home/seewon/mimic3-benchmarks/data/transfer/vital_triage.csv')

In [ ]:
df = df[['stay_id', 'pain', 'chiefcomplaint', 'rhythm', 'charttime']]

In [ ]:
df['pain'] = pd.to_numeric(df['pain'], 'coerce')

In [16]:
df.to_csv('/home/seewon/mimic3-benchmarks/data/transfer/vitals.csv')

In [47]:
numerics = pd.read_csv('/home/seewon/mimic3-benchmarks/data/transfer/numerics.csv')

In [ ]:
numerics = df[['stay_id', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'acuity', 'charttime']]

In [56]:
numerics = numerics.merge(df[['stay_id', 'pain', 'charttime']], on=['stay_id', 'charttime'], how='outer')

In [57]:
numerics.to_csv('/home/seewon/mimic3-benchmarks/data/transfer/numerics.csv', index=False)